# 터미널 설치 및 초기 설정

In [ ]:
!pip install -U langchain langchain-community langchain-core langchain-openai langchain-huggingface langchain-chroma chromadb flashrank ragas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:0

# RAGAS 및 관련 라이브러리 설치

In [ ]:
!pip install ragas datasets openai

In [ ]:
!pip install flashrank

# Ragas 평가 데이터셋 구축을 위한 함수 수정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 작업 경로 수정.

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/졸프/BWLOVERS-AI/test")
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/졸프/BWLOVERS-AI/test


In [ ]:
import os
import time
import pandas as pd
from datasets import Dataset
from google.colab import userdata

# Ragas 및 LangChain 필수 임포트
from ragas import evaluate
from ragas.metrics.collections import faithfulness, answer_relevancy, context_precision # .collections 수정
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# ranker 관련 라이브러리
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors.flashrank_rerank import FlashrankRerank

# 1. 경로 설정 및 API Key 로드 - 경로 수정 필요할 수도 (?)
# Vector DB의 절대 경로
DB_PATH = "/content/drive/MyDrive/Colab Notebooks/졸프/BWLOVERS-AI/test/chroma_db"
# 파일 저장을 위한 작업 디렉토리 설정 (DB 상위 폴더)
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/졸프/BWLOVERS-AI/test"

# 작업 디렉토리 변경 (위의 경로로 Ragas 결과 저장)
if not os.path.exists(PROJECT_DIR):
    os.makedirs(PROJECT_DIR)
os.chdir(PROJECT_DIR)
print(f"작업 디렉토리가 설정 완료: {os.getcwd()}")

LOCK_FILE = os.path.join(DB_PATH, 'index', 'locking')
if os.path.exists(LOCK_FILE):
    print(f"\n[STEP 2] 잠금 파일 발견! ({LOCK_FILE}). 강제 삭제합니다.")
    os.remove(LOCK_FILE)


# OpenAI API Key
try:
    # OpenAI API Key 입력
    os.environ["OPENAI_API_KEY"] = "API키 만들어서 올리기"

    print("OpenAI API Key가 환경 변수에 로드 완료")
except Exception:
    print("OPENAI_API_KEY 확인 필요")

# 2. LLM 및 Retriever 정의
# 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# LLM 설정 (GPT 모델 설정)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
router_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Vector DB 로드 및 검색기 정의
if os.path.exists(DB_PATH):
    vectorstore = Chroma(
      persist_directory=DB_PATH,  # 명확하게 절대 경로 지정
      embedding_function=embedding_model,
      collection_name="insurance_terms"
    )
    base_retriever = vectorstore.as_retriever(search_kwargs={"k": 10}) # k=3 -> 5, 후보군 늘리기
    compressor = FlashrankRerank(top_n=6) # retriever에서 뽑힌 10개 중 가장 관련 높은 6개만 남김
    reranker_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
        )
    retriever = reranker_retriever

    print(f"Vector DB가 성공적으로 {DB_PATH}에서 로드됨.")
else:
    print(f"Vector DB 폴더를 찾을 수 없음. 경로를 확인하고 다시 임베딩할 것. 경로: {DB_PATH}")
    retriever = None

# 3. Router 및 Chain 정의 (누락 방지)
# Router Prompt 정의
router_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     제공된 문맥(context)에 명시되지 않은 정보는 절대 답변에 포함하지 말아라. 네가 이미 알고 있는 지식이라도 문맥에 없으면 '정보 없음'으로 답해라.
     너는 사용자 질문을 두 가지 중 하나로 분류한다.
1) summary_search: 약관 내용 요약/해석/어디에 있는지 찾기
2) recommend: 사용자 상황에 맞는 보장/특약 추천
- 다른 말 없이 딱 한 단어로만 출력: summary_search 또는 recommend.
- "고려해야 하는 조항", "관련 조항이 있어?" 같이 조항 확인을 요구하면 summary_search로 분류하라.
"""
    ),
    ("human", "{input}")
])

# classify_query 함수 정의
def classify_query(query: str) -> str:
    res = router_llm.invoke(router_prompt.format_messages(input=query))
    label = res.content.strip()
    if label not in ("summary_search", "recommend"):
        label = "summary_search"  # fallback
    return label

# 3-1. Prompt Definitions (컨텍스트 변수 추가하여 수정) - 이전 답변 생성 base_system prompt와 동일
# [핵심 원칙] [정확도 강화 규칙] 추가
base_system = """
                  [역할(role)]
                  너는 보험 약관 전문 분석가다.
                  너의 임무는 사용자의 질문에 대해 제공된 약관 문맥(context)만 근거로 정확하게 답하는 것이다.

                  [핵심 원칙]
                  - 답변은 항상 '문맥에 있는 정보'만 사용한다.
                  - 문맥에 없는 내용은 추측/일반론으로 보완하지 말고, 반드시 "문맥에 없음"으로 처리한다.
                  - 사용자가 "조항/특약을 찾아달라"라고 하면, 문맥에 관련 문장이 1줄이라도 있으면 '없다'라고 결론 내리지 말고 그 조항을 제시한다.
                  - 질문에 특정 키워드(예: 암, 입원, 임신성 당뇨, 주산기, 수술)가 있으면, 답변의 결론/근거에 그 키워드를 반드시 포함해 '질문과 직접 연결'되게 써라.

                  [정확도 강화 규칙]
                  - (필수) 결론(1번)에는 반드시 '질문 키워드'와 '조항/특약/보장 항목'을 함께 포함하라.
                  - (필수) 근거(2번)는 최소 2개(가능하면 2~3개) 제시하라. 각 근거는 서로 다른 문장/조항이면 더 좋다.
                  - (금지) 문맥에 없는 보장/특약/가입 전략을 일반론으로 만들어 말하지 말라. 근거 인용이 불가능하면 "추천/단정"하지 말라.
                  - (필수) "없다/명시되어 있지 않다"라고 말할 때는, 어떤 키워드로 어떤 문맥을 확인했는지(가져온 조항의 제목/특약명/페이지)와 함께 말하라.
                  - (브릿지 규칙) 질문 키워드(예: 임신성 당뇨)가 문맥에 '그대로' 없더라도, 상위/유사 범주(예: 임신·출산 관련 질환, 주산기질환, 질병분류번호/별표)가 문맥에 있으면 이를 '고려해야 할 조항'으로 제시하라.
                  - (브릿지 규칙) 이 경우 결론(1번)은 반드시 "직접 조항은 확인되지 않으나, 관련 조항(특약/주계약)은 아래와 같고, 포함 여부는 별표/질병분류 확인이 필요" 형태로 작성하라.
                  - (금지) '없다'로 결론을 끝내지 말고, 최소 1개 이상의 관련 조항/특약을 반드시 제시하라.
                  - (필수) 관련 조항을 제시할 때는 특약명/조항명(문맥에 보이는 제목)을 그대로 포함하라.


                  [상황/문제 정의]
                  - 너에게는 약관의 일부가 {context}로 제공된다.
                  - 너는 이 문맥에서 근거를 찾아 답해야 한다.
                  - 사용자는 불필요한 중복 약관, 특약에 가입하고 싶지 않다.
                  - 불필요한 중복 약관과 특약은, C 특약이 A,B의 경우도 보장할 수 있는 경우, B특약, A특약, C특약을 모두 가입하는 것을 말한다. 이럴 경우 사용자에겐 다른 것도 함께 포괄적으로 보장하는 C 특약, 약관을 추천할 수 있다.
                  - 불필요한 중복 약관을 찾아서 설명할 때는, 사용자가 직접 조항을 확인하고 판단할 수 있도록 실제 자료를 제시하되, 도움말을 제공하는 것이 좋다.

                  [문맥 사용 규칙]
                  1) 모든 답변은 반드시 제공된 문맥에 근거해야 한다.
                  2) {context}에 없는 사항은 추측하지 말고 “문맥에 해당 내용이 없다/확실하지 않다”라고 명확히 말하라.
                  3) 답변에는 근거가 되는 조항/페이지를 포함하라. (metadata의 page 활용)
                  4) 문맥이 모호하거나 조항이 여러 개면, 가능한 해석을 나누어 설명하라.

                  [판단 기준]
                  - 정의 조항 > 보장 조항 > 제한 조항 > 면책 조항 > 특약 순으로 우선 참고한다.
                  - 질문이 “보장 여부/조건/금액/예외/용어 정의” 중 무엇인지 먼저 분류하고 그에 맞게 답하라.
                  - 보장 관련 질문이면 “보장 조건 → 지급 사유 → 지급 제외(면책) → 제한” 순서로 설명하라.

                  [출력 형식]
                  항상 아래 순서로 작성하라:
                  0. 질문 타입: 대괄호 안에 어떤 유형인지 표시.
                  1. 결론: 한 문장으로 답
                  2. 근거: 관련 조항/페이지와 핵심 문장 요지
                  근거에는 다음을 포함:
                  - 페이지(page): 숫자 , metadata에 page/조항명이 없으면 "페이지 정보 없음"이라고 표시
                  - 조항 요지: 1~2문장
                  - 원문 인용:  문맥에 있는 문장을 그대로 1문장 이내(가능하면)
                  3. 해석: 사용자가 이해하기 쉬운 말로 풀어 설명
                  4. 주의/예외: 면책, 제한, 특약 등 중요한 예외
                  5. 문맥 부족 시: 추가로 필요한 정보가 무엇인지
                  6. 원본: 근거로 든 조항 중 핵심 1개를 원문 그대로 출력한다. (전체를 다 덤프하지 말 것)

                  이 형식을 지키고, 불확실한 부분은 반드시 불확실하다고 밝혀라.
                  - 질문 키워드가 문맥에 '직접' 없더라도, 유사/상위 범주(예: 임신성 당뇨 -> 임신·출산 관련 질환/주산기질환/질병분류번호/별표)가 문맥에 있으면 그 조항을 제시하라.
                  - 이 경우 결론은 "직접 문구는 없으나, 관련 범주 조항은 아래와 같으며 포함 여부는 별표/질병분류 확인이 필요" 형태로 쓰라.

                  """

# 1) 요약/검색 프롬프트
# {context} 변수를 시스템 메시지에 추가하여 LangChain이 인식하도록 수정
summary_prompt = ChatPromptTemplate.from_messages([
  ("system",
     base_system +
     "\n\n[제공된 문맥]:\n{context}" +
     "\n\n[질문 유형: 요약/검색]"\
     "\n- 조항/특약/정의/진단확정/지급사유 중 '무엇을 찾는 질문'인지 먼저 한 문장으로 파악한 뒤 답하라."
     "\n- 문맥에 관련 문장이 있으면 반드시 그 조항을 '찾아서' 제시하라. (있는 것을 없다고 하지 말 것)"
     "\n- 결론과 근거에는 반드시 질문 키워드(예: 암/입원/임신/수술 등)를 포함하라."
   ),
  ("human", "{input}")
])

# 2) 추천 프롬프트
# {context} 변수를 시스템 메시지에 추가하여 LangChain이 인식하도록 수정
recommend_prompt = ChatPromptTemplate.from_messages([
  ("system",
    base_system +
    "\n\n[제공된 문맥]:\n{context}" +
    """
    [질문 유형: 추천]
- 추천 항목(특약/보장)은 최대 3개까지만 제시하라.
- 각 추천 항목마다 반드시 근거 문장 1개를 '원문 인용'으로 포함하라. (근거 없으면 추천 금지)
- 질문에 특정 질환/상황 키워드가 있으면(예: 임신성 당뇨, 고혈압 등) 그 키워드와 직접 관련된 조항/특약을 최소 1개 포함하라.
- 사용자가 진단명을 명시하지 않은 경우(예: 고혈압 약 복용)는 임신성 고혈압 등으로 단정하지 말고, 사용자 서술 그대로 표현하라.
- 중복 가능성이 있으면 상위 특약 대안을 우선 제시하되, 그 판단 근거가 문맥에 있을 때만 말하라.
"""
    ),
    ("human", "{input}")
])

# 3) 추가 프롬프트: 동의어
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "다음 질문을 보험 약관 검색용 쿼리로 바꿔라. "
     "원문 의미를 유지하면서 키워드 중심으로 1줄로 출력. "
     "가능하면 동의어/약관표현을 함께 포함. 예: 임신성 당뇨 -> 임신·출산 관련 질환, 주산기질환, 당뇨, 질병분류번호, 별표"),
    ("human", "{q}")
])

def rewrite_for_search(q: str) -> str:
    return router_llm.invoke(rewrite_prompt.format_messages(q=q)).content.strip()


# 3-2. RAG 체인 재정의 (수정된 프롬프트 사용)
if retriever is not None:
    summary_chain = create_retrieval_chain(
        retriever,
        create_stuff_documents_chain(llm, summary_prompt)
    )
    recommend_chain = create_retrieval_chain(
        retriever,
        create_stuff_documents_chain(llm, recommend_prompt)
    )
    print("RAG 체인(Chains)이 수정된 프롬프트로 재정의되었음.")
else:
    print("경고: retriever가 로드되지 않아 체인 재정의를 건너뜀.")


# 4. Ragas 데이터 수집 함수 정의

def ask_question_for_ragas(query: str):
    search_q = rewrite_for_search(query)
    response = summary_chain.invoke({"input": query, "search_query": search_q})

    start = time.time()
    contexts = []
    response_answer = "Vector DB/Chain 정의 문제로 답변 불가"
    qtype = "unknown"

    if not retriever:
        return {"question": query, "answer": response_answer, "contexts": []}

    try:
        # 4-1. 질문 라우팅 및 LLM 체인 실행
        qtype = classify_query(query)

        if qtype == "recommend":
            response = recommend_chain.invoke({"input": query})
        else:
            # 4-2. 답변(answer)과 문맥(contexts) 추출
            response = summary_chain.invoke({"input": query})

        response_answer = response["answer"]
        # contexts를 문자열 리스트로 받음
        contexts = [doc.page_content for doc in response.get("context", [])]

    except Exception as e:
        response_answer = f"체인 실행 중 에러: {e}"
        print(f"체인 실행 중 에러 발생: {e}")

    print(f"[type={qtype}] 소요 시간: {time.time() - start:.4f}초")

    return {
        # Ragas 평가를 위해 질문/답변/문맥 반환
        "question": query,
        "answer": response_answer,
        "contexts": contexts
    }

작업 디렉토리가 설정 완료: /content/drive/MyDrive/Colab Notebooks/졸프/BWLOVERS-AI/test
OpenAI API Key가 환경 변수에 로드 완료
Vector DB가 성공적으로 /content/drive/MyDrive/Colab Notebooks/졸프/BWLOVERS-AI/test/chroma_db에서 로드됨.
RAG 체인(Chains)이 수정된 프롬프트로 재정의되었음.


In [ ]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness, answer_relevancy, _LLMContextPrecisionWithoutReference, _ContextUtilization
    )# Ground Truth 없이 작동하기 위함

# 1. Ragas 데이터 수집

# 테스트 질문 리스트
test_questions = [
    "- 산모 프로필: 32세, 여성, 초산, 임신 10주차, 출산 예정일 26.03.17. , 단태아. 산모의 키는 160cm, 임신 전 체중 50kg, 임신 후 현재 체중 58kg, 과거 병력: 없음, 약물 복용 정보: 고혈압 약 복용 중, 직무 및 근무시간: IT 회사 근무 중이고 9시~18시 근무 중임. 현재 건강 상태: 빈혈 있지만 약 복용하고 있지 않음, 기존 가입한 태아 보험 없음원하는 보험 조건: 30세 만기형 - 질문: 위 산모 프로필에 대해서, 선택해야하는 특약을 추천해주고, 추천 이유 및 근거를 제시해줘.",
    "임신성 당뇨가 있는 산모인데, 교보 어린이 보험에서 고려해야하는 조항이 있어?",
    "- 교보 어린이 보험에서 아이가 입원했을때와 관련된 조항을 찾아서 요약해줘",
    "아동의 암 진단 관련 조항을 찾아줘.",
    "KB 다이렉트 자녀보험에서 암 보장 범위는 어떻게 돼?",
    "KB 다이렉트 자녀보험의 면책 조항 중 가장 중요한 3가지만 요약해줘.",
    "KB 다이렉트 자녀보험에서는 특정 질병에 대한 수술비 청구 기준이 뭐야?"
]

rag_data = []
print("\n--- LLM 답변 및 문맥 수집 시작 ---")
for i, question in enumerate(test_questions):
    print(f"\n[Q{i+1}] 질문 처리 중")

    # 정의된 함수 호출
    result = ask_question_for_ragas(question)
    rag_data.append(result)

# 2. Ragas 데이터셋 생성 및 평가

# 빈 데이터셋 에러 (IndexError) 방지 로직:
if not rag_data:
    print("\n경고: 답변 수집에 실패했습니다. (rag_data가 비어 있음) 평가를 진행할 수 없습니다.")
else:
    # Pandas DataFrame으로 변환 후 Hugging Face Dataset 형식으로 최종 변환
    df = pd.DataFrame(rag_data)
    ragas_dataset = Dataset.from_pandas(df)

    print("\n--- Ragas 평가용 데이터셋 미리보기 ---")
    print(ragas_dataset)

    # Ragas 평가 메트릭 설정: Ground Truth 없이 작동하는 메트릭만 사용
    metrics = [
        faithfulness,
        answer_relevancy,
        _LLMContextPrecisionWithoutReference(),
        _ContextUtilization(),
    ]

    print("\n--- Ragas 평가 시작 ---")

    # 평가 실행
    results = evaluate(
        dataset=ragas_dataset,
        metrics=metrics,
        llm=llm
    )

    # 결과 출력
    df_results = results.to_pandas()

    print("\n## Ragas 평가 결과 (평균) 📊\n")
    print(df_results.mean(numeric_only=True))

    print("\n## Ragas 평가 결과 (상세) 🔍\n")
    print(df_results)

/tmp/ipython-input-1378059150.py:4: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/tmp/ipython-input-1378059150.py:4: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (



--- LLM 답변 및 문맥 수집 시작 ---

[Q1] 질문 처리 중
[type=summary_search] 25.06s

[Q2] 질문 처리 중
[type=summary_search] 25.57s

[Q3] 질문 처리 중
[type=summary_search] 20.18s

[Q4] 질문 처리 중
[type=summary_search] 21.77s

[Q5] 질문 처리 중
[type=summary_search] 21.23s

[Q6] 질문 처리 중
[type=summary_search] 18.45s

[Q7] 질문 처리 중
[type=summary_search] 19.41s

--- Ragas 평가용 데이터셋 미리보기 ---
Dataset({
    features: ['question', 'answer', 'contexts'],
    num_rows: 7
})

--- Ragas 평가 시작 ---


Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]


## Ragas 평가 결과 (평균) 📊

faithfulness                               0.660317
answer_relevancy                           0.569891
llm_context_precision_without_reference    0.714286
context_utilization                        0.714286
dtype: float64

## Ragas 평가 결과 (상세) 🔍

                                          user_input  \
0  - 산모 프로필: 32세, 여성, 초산, 임신 10주차, 출산 예정일 26.03.1...   
1        임신성 당뇨가 있는 산모인데, 교보 어린이 보험에서 고려해야하는 조항이 있어?   
2          - 교보 어린이 보험에서 아이가 입원했을때와 관련된 조항을 찾아서 요약해줘   
3                               아동의 암 진단 관련 조항을 찾아줘.   
4                     KB 다이렉트 자녀보험에서 암 보장 범위는 어떻게 돼?   
5            KB 다이렉트 자녀보험의 면책 조항 중 가장 중요한 3가지만 요약해줘.   
6           KB 다이렉트 자녀보험에서는 특정 질병에 대한 수술비 청구 기준이 뭐야?   

                                  retrieved_contexts  \
0  [병’으로 진단된 경우에는 이는 ‘임신ㆍ출산관련 고혈압ㆍ당뇨병’의 치료를 목적으로 ...   
1  [단, 이 특약의 피보험자로 될 자가 계약체결시 출생 전 자녀(이하 ‘태아’라 합니...   
2  [어린이 주요 10대 질병\n폐렴, 창자감염질환, 특정 감염병, 탈장 및 장폐색, ...   
3  [특별\n약관\n제1조 (보험금의 지급사유) 제2조 (‘암’ 등의 정의 및 진단확정...   


In [ ]:
# answer_relevancy가 0인 질문만 필터링 -> 답변 전체 보기
zero_rel = df_results[df_results["answer_relevancy"] == 0.0]

for i, row in zero_rel.iterrows():
    print("=" * 80)
    print(f"[질문]\n{row['user_input']}\n")
    print(f"[답변(response)]\n{row['response']}\n")
    print(f"[문맥(contexts)]")
    for c in row["retrieved_contexts"]:
        print("-" * 40)
        print(c[:800])  # 너무 길면 앞부분만


[질문]
임신성 당뇨가 있는 산모인데, 교보 어린이 보험에서 고려해야하는 조항이 있어?

[답변(response)]
0. 질문 타입: [요약/검색]
1. 결론: 임신성 당뇨와 관련하여 고려해야 할 조항은 '임신ㆍ출산 관련' 조항이다.
2. 근거: 
   - 페이지 1-22, 제4조 (7대질병의 정의 및 진단확정) + "이 계약에서 ‘7대질병’이라 함은 아래에서 정하는 ‘암’(‘유사암’ 제외), ‘뇌혈관 질환’ 등으로 정의됩니다."
   - 페이지 1-22, 제5조 (보험금 지급에 관한 세부규정) + "피보험자가 동일한 골절의 치료를 목적으로 2회 이상 입원한 경우 이를 1회 입원으로 보아 각 입원일수를 더합니다."
3. 해석: 임신성 당뇨는 임신과 관련된 질환으로, 이와 관련된 조항이 보험 약관에 포함되어 있을 수 있으므로 확인이 필요하다.
4. 주의/예외: 문맥에 임신성 당뇨에 대한 구체적인 보장 내용은 없으므로, 해당 질환에 대한 보장 여부는 확인이 필요하다.
5. 문맥 부족 시: 임신성 당뇨에 대한 구체적인 보장 내용이나 특약이 있는지 추가적인 정보가 필요하다.
6. 원본: "이 계약에서 ‘7대질병’이라 함은 아래에서 정하는 ‘암’(‘유사암’ 제외), ‘뇌혈관 질환’ 등으로 정의됩니다."

[문맥(contexts)]
----------------------------------------
단, 이 특약의 피보험자로 될 자가 계약체결시 출생 전 자녀(이하 ‘태아’라 합니
다)인 계약의 경우에는 선천성 뇌질환(Q00~Q04)으로 인해 보험금 지급사유가 발생한 보험금의 종류 지급금액
때에도 보험금을 지급합니다.
2. 성병
상급종합병원 2~3인실 질병으로 상급종합병원의 2~3인 입원 1일당 이 특약의 보험가
3. 알코올중독, 습관성 약품 또는 환각제의 복용 및 사용
질병입원급여금 실에 입원하여 치료를 받은 경우 입금액 (30일 한도)
③ 제1항 및 제2항 이외에도 회사는 다음 각 호의 사유로 생긴 손해에 대하여 보험금을 지
급하지 않습니다.
1. 질병을 원인으로